In [39]:
#ensembling
from mlxtend.classifier import StackingCVClassifier
import numpy as np
import pandas as pd
import sys, os, glob
import imp
import seaborn as sns
sys.path.append('./src/')
import common_utils,my_ensembler, feature_utils, defines, model_utils, my_bert
from sklearn.neighbors import KNeighborsClassifier
from sklearn_crfsuite import scorers, CRF
from sklearn.pipeline import Pipeline
import json
from operator import itemgetter
from sklearn_crfsuite.utils import flatten
from sklearn_crfsuite.metrics import sequence_accuracy_score
from sklearn.linear_model import LogisticRegression
import scipy
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import classes
import pickle
import matplotlib.pyplot as plt
import copy
import segeval as se

In [2]:
dir_name='reparse'
seq_len=15
step=15
cv_split_name='cv_splits_10'
merged_str='scaled.binary0.50_merged'
doc_indices=np.arange(1,81)
doc_indices
test_doc_num=8
seed=42

In [3]:
dir_name='reparse'
docs_map_name = 'scaled.binary.5_stop_docs_map.json'
crf_params_name='.802_21.07_07:26_best_params.json'
ens_crf_db_name   = 'ensemble_crf_cv_db_all'
ens_lg_db_name = 'ensemble_lg_cv_db_all'
cv_crf_bert_name = 'cv_crf_bert_all'
per_par=True
seq_len=3
step=3

### Open 1 sentence db

In [8]:
sent_db = pd.DataFrame(common_utils.load_db(dir_name,"01_sent_db")['is_nar'])

Opened /scratch/sim_reg6/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/reparse/01_sent_db.csv,  index False


In [73]:
sent_db

,is_nar
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
608,0.0
609,0.0
610,0.0
611,0.0


In [75]:
imp.reload(classes)
my_se =classes.MySegVal()
seg_from_class=my_se.get_segment_list( sent_db['is_nar'].tolist())
seg_from_class

MySegVal init called


[21,
 43,
 27,
 24,
 41,
 19,
 44,
 11,
 53,
 22,
 1,
 59,
 16,
 5,
 8,
 3,
 41,
 8,
 7,
 24,
 23,
 20,
 7,
 10,
 1,
 8,
 67]

In [116]:
y_pred={}
y_pred['a']=[0,0,0,0,0,0,1,1,1,1,1,1]
y_pred['b']=[0,0,0,0,0,0,0,0,0,0,0,0]
y_pred['c']=[0,0,0,0,0,1,1,1,1,1,1,1]
y_pred['d']=[0,1,1,1,1,1,0,0,0,0,0,0]
y_pred['e']=[0,0,0,0,0,1,0,1,1,1,1,1]

In [133]:
imp.reload(classes)
my_se=classes.MySegEVal(n_t=2)
for k,v in y_pred.items():    
    res=my_se.get_scores(y_pred['a'],v)
    print("="*8)
    print("{} {}\n{} {}".format('a',y_pred['a'],k,v))
    for i,j in res.items():
        if isinstance(j,dict):
            print("{}".format(i))
            for m,n in j.items():
                print("\t{}:{}".format(m,n))
        else:
            print("{}:{}".format(i,j))

MySegEVal init called
a [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
a [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
f1:1
recall:1
precision:1
b_sim:1
s_sim:1
b_stat
	count_edits:0
	additions:[]
	substitutions:[]
	transpositions:[]
	full_misses:[]
	boundaries_all:2
	matches:[1]
	pbs:11
	boundary_types:frozenset({1})
a [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
b [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
f1:0
recall:0
precision:0
b_sim:0
s_sim:0.9090909090909090909090909091
b_stat
	count_edits:1
	additions:[Addition(type=1, side='a')]
	substitutions:[]
	transpositions:[]
	full_misses:[1]
	boundaries_all:1
	matches:[]
	pbs:11
	boundary_types:frozenset({1})
a [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
c [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]
f1:1
recall:1
precision:1
b_sim:0.5
s_sim:0.9545454545454545454545454545
b_stat
	count_edits:0.5
	additions:[]
	substitutions:[]
	transpositions:[Transposition(start=4, end=5, type=1)]
	full_misses:[1, 1]
	boundaries_all:2
	matches:[]
	pbs:11
	boundary_types:frozenset({1})
a [0, 0, 0, 0

In [101]:
y['true_seg']=my_se.get_segment_list(y['true'])
y['pred_seg']=my_se.get_segment_list(y['pred'])

In [102]:
y['true_seg'],y['pred_seg']

([2, 3, 3], [3, 2, 3])

In [103]:
conf_matrix=se.boundary_confusion_matrix(y['true_seg'],y['pred_seg'])

In [104]:
se.precision(conf_matrix)

Decimal('1')

In [105]:
se.recall(conf_matrix)

Decimal('1')

In [106]:
sese.fmeasure(conf_matrix)

Decimal('1')

In [32]:
y=[0,0,0,0,1,1,1,1,1,0,0,1,1,0,1,0,0,0]

In [35]:
y_db  = pd.DataFrame(y,columns =['is_nar'])
y_db

,is_nar
0,0
1,0
2,0
3,0
4,1
5,1
6,1
7,1
8,1
9,0


In [37]:
y_db.items()

<generator object DataFrame.items at 0x2ba9d16fddd0>

In [59]:
seg_size_list,seg_size_list_2

([21,
  43,
  27,
  24,
  41,
  19,
  44,
  11,
  53,
  22,
  1,
  59,
  16,
  5,
  8,
  3,
  41,
  8,
  7,
  24,
  23,
  20,
  7,
  10,
  1,
  8,
  67],
 [1,
  21,
  43,
  27,
  24,
  41,
  19,
  44,
  11,
  53,
  22,
  1,
  59,
  16,
  5,
  8,
  3,
  41,
  8,
  7,
  24,
  23,
  20,
  7,
  10,
  1,
  66])

In [68]:
l1=[3,4,3,5,3]
l2=[4,3,1,1,1,6]

In [70]:
se.boundary_similarity(l1,l2,n_t=2)

Decimal('0.2')

In [71]:
se.segmentation_similarity(l1,l2,n_t=2)

Decimal('0.7647058823529411764705882353')